# Tensor

In [4]:
import torch
import torch.nn as nn

## 创建tensor
torch.tensor()

In [5]:
import numpy as np

In [6]:
d = np.array([1,2,3])

In [7]:
td = torch.tensor(d)
td

tensor([1, 2, 3], dtype=torch.int32)

In [9]:
id(td)

2132550913984

### avoid copy

In [12]:
td1 = torch.tensor([1,2,3])
print(f'id(td1)={id(td1)}')
td2 = td1.detach()
print(f'id(td2)={id(td2)}')
td3 = torch.as_tensor(td2)
print(f'id(td3)={id(td3)}')

id(td1)=2132550913192
id(td2)=2132550991944
id(td3)=2132550991944


### dtype
tensor的默认类型是`torch.float`,有几种设置tensor类型的方式

In [24]:
td1 = torch.tensor([1,2,3], dtype=torch.int32)
td1,id(td1)

(tensor([1, 2, 3], dtype=torch.int32), 2132550992808)

In [23]:
td2 = td1.type(torch.bool)
td2,id(td2)

(tensor([True, True, True], dtype=torch.bool), 2132550994320)

In [18]:
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

In [22]:
td3 = td1.type(Tensor)
td3,id(td3)

(tensor([1., 2., 3.], device='cuda:0'), 2132550994824)

可以使用Tensor.to()进行类型转换

In [27]:
td4 = td1.to(torch.float)
td4,id(td4)

(tensor([1., 2., 3.]), 2132550994104)

## tensor转换为其他类型

### 导出python number
torch.Tensor.item()

In [14]:
td = torch.tensor([3.14])
pi = td.item()
pi

3.140000104904175

### numpy

In [26]:
td1 = torch.tensor([1,2,3], dtype=torch.int32)
npd = td.numpy()
npd

array([3.14], dtype=float32)

## in-place version
in-place operation在pytorch中是指改变一个tensor的值的时候，不经过复制操作，而是直接在原来的内存上改变它的值。可以把它成为原地操作符。

在pytorch中经常加后缀“_”来代表原地in-place operation，比如说.add_()。python里面的+=，*=也是in-place operation。

In [28]:
td = torch.tensor([1,2,3])
td1 = torch.tensor([4,5,6])
print(f"id(td)={id(td)}")
td = td.add_(td1)
print(f"id(td)={id(td)}")


id(td1)=2132551143928
id(td)=2132551143928


In [30]:
td = torch.tensor([1,2,3])
td1 = torch.tensor([4,5,6])
print(f"id(td)={id(td)}")
td = td.add(td1)
print(f"id(td)={id(td)}")

id(td)=2132551143928
id(td)=2132551144360


## tensor shape相关

#### tensor shape
```
a = torch.randn(10,32,32,3)
这个a可以解释为保存batch=10，size=32*32，channel为3的图像数据的变量。
a有4个维度，我们将第一个维度大小为10，称为最高的维度，第四个维度大小为3，称为最低的维度，较低的维度的数据是“放在一起的/内存相连的”
```

In [2]:
a = torch.randn(10,32,32,3)
a.dim()

4

#### view()
view(a,b,c...)将根据第一个参数将内存划分a大块，然后每大块划分b中块，然后每中块划分b小块，依次类推...<br>
如何给view()传入参数呢？要根据tensor存放数据方式对应的物理意义来决定。<br>
例如tensor a存放了24个成员，包括了8个像素，每个像素用3通道数据表示，也就是我们将像素数量对应的维度定义为了高维度，通道数对应的维度定义为了低维度，那么通道数据是连在一起的，故该操作为：
```
a = a.view(8,3)
```

### Demo1

In [11]:
a = torch.range(1,9)
print(a)
a = a.view(3,3)
print(a)
print(a.size())

C:\ProgramData\Anaconda3\envs\dp_pytorch11_GPU\lib\site-packages\ipykernel_launcher.py:1: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  """Entry point for launching an IPython kernel.


tensor([1., 2., 3., 4., 5., 6., 7., 8., 9.])
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])
torch.Size([3, 3])


In [17]:
b = a.repeat(2,1,1)
print(b,b.size())
print(b.is_contiguous())
b = b.permute(1,2,0)
print(b,b.size())
print(b.is_contiguous())
b = b.contiguous()

tensor([[[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]]) torch.Size([2, 3, 3])
True
tensor([[[1., 1.],
         [2., 2.],
         [3., 3.]],

        [[4., 4.],
         [5., 5.],
         [6., 6.]],

        [[7., 7.],
         [8., 8.],
         [9., 9.]]]) torch.Size([3, 3, 2])
False


### permute维度交换

In [9]:
import torch
from torch.autograd import Variable
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

In [19]:
a = torch.randn(32,32,3)
a.size()

torch.Size([32, 32, 3])

In [20]:
b = Variable(a.type(Tensor))
b[0,0]

tensor([ 0.1657, -1.1956,  1.2423], device='cuda:0')

In [21]:
b = b.permute(2,1,0)
b.size()

torch.Size([3, 32, 32])

In [22]:
b[:,0,0]

tensor([ 0.1657, -1.1956,  1.2423], device='cuda:0')

### Demo2_view与permute的区别
- view: view()操作的tensor必须是contiguous的，也就是按照内存地址的顺序根据输入的维度进行划分，但数据的存放顺序是完全没有改变的，仍然在连续的一块内存。
- permute: permute()是更灵活的transpose()，用于维度的交换，该操作完成后，数据的不再按照原来地址存放。

In [18]:
a = torch.range(1,6).view(2,3)
print(a)

C:\ProgramData\Anaconda3\envs\dp_pytorch11_GPU\lib\site-packages\ipykernel_launcher.py:1: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  """Entry point for launching an IPython kernel.


tensor([[1., 2., 3.],
        [4., 5., 6.]])


In [22]:
b = a.view(3,2)
print(b.is_contiguous())
print(b)

True
tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])


In [21]:
c = a.permute(1,0)
print(c.is_contiguous())
print(c)

False
tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])


### Demo3


In [31]:
pre = torch.randn(10,3,13,13,8)
x = pre[...,0]
print(x.shape)

torch.Size([10, 3, 13, 13])


In [40]:
g = 13
grid_x = torch.arange(g)
print(grid_x, grid_x.shape)
grid_x = grid_x.repeat(g, 1)
print(grid_x, grid_x.shape)
grid_x = grid_x.view([1, 1, g, g]).type(torch.float)
print(grid_x.shape)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]) torch.Size([13])
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]]) torch.Size([13, 13])
torch.Size([1, 1, 13, 13])


In [39]:
res_x = x + grid_x
print(res_x.shape)

torch.Size([10, 3, 13, 13])


## tensor索引
对于多维的tensor来说，索引参数的size必须一样，如果不一样那么其size应该为1。

### 通过比较运算符获取mask
这种方式获取的mask与tensor具有相同size

In [3]:
x = torch.randn(4,3,5,5,8)
msk = x>0.5
print(msk.size())

torch.Size([4, 3, 5, 5, 8])


### 多维tensor提取
x的size表示为(batch,anchor,width,width,channels)，如果希望根据特定条件，取出4个(msk的长度就是4)满足anchor条件、width条件、height条件的所有channel，则代码如下：

In [21]:
x = torch.randn(4,3,5,5,8)
msk1 = torch.tensor([0,1,0,1])
msk2 = torch.tensor([1,0,1,2])
msk3 = torch.tensor([0,4,1,2])
msk4 = torch.tensor([1,4,3,0])
y    = x[msk1,msk2,msk3,msk4]
'''
for i in range(len(msk1)):
    res = x[msk1[i],msk2[i],msk3[i],msk4[i]]
'''
print(y,y.size())

tensor([[ 1.6896,  0.3553,  0.4264, -0.3482, -1.6622, -1.3962, -1.3688,  0.1051],
        [-1.5447, -0.8086,  1.0541, -1.2533, -1.3357, -1.0364,  0.3644, -0.3640],
        [ 0.2106, -0.0392, -2.4183, -1.2803,  1.0036, -0.0490, -1.3218, -0.3913],
        [-0.0868, -1.2071,  1.4577, -0.3203,  0.0091,  1.3113, -0.6703, -0.2029]]) torch.Size([4, 8])


In [22]:
msk = torch.ByteTensor(4,3,5,5).fill_(0)
msk[msk1,msk2,msk3,msk4] = 1
print(msk)

tensor([[[[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 1, 0, 0, 0],
          [0, 0, 0, 1, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]]],


        [[[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 1]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [1, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]]],


        [[[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          

### mask与tensor具有相同size

In [2]:
x = torch.randn(3,5)
msk = torch.ByteTensor(3,5).fill_(1)
print(x)
print(msk)

tensor([[-0.0105, -1.4868,  1.3388, -1.2086,  0.0069],
        [ 0.4063, -0.7230, -0.9968, -0.4063, -0.6198],
        [ 0.2121,  1.7841,  0.0084, -1.0090, -0.1287]])
tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]], dtype=torch.uint8)


In [3]:
y = x[msk]
y

tensor([-0.0105, -1.4868,  1.3388, -1.2086,  0.0069,  0.4063, -0.7230, -0.9968,
        -0.4063, -0.6198,  0.2121,  1.7841,  0.0084, -1.0090, -0.1287])

### max

In [2]:
a = torch.randn(5,4)
a

tensor([[ 1.1408, -0.5380, -0.3321, -1.2923],
        [-0.5694,  0.5125,  0.2685, -1.6011],
        [ 0.8303, -1.1273,  1.2312,  1.4933],
        [-0.0817,  0.3833,  1.1367, -0.5969],
        [ 0.1217,  0.2332,  1.9962,  0.0047]])

In [6]:
b = a.max(1)
print(type(b))
b[0]

<class 'torch.return_types.max'>


tensor([1.1408, 0.5125, 1.4933, 1.1367, 1.9962])

In [7]:
b

torch.return_types.max(
values=tensor([1.1408, 0.5125, 1.4933, 1.1367, 1.9962]),
indices=tensor([0, 1, 3, 2, 2]))

## GPU

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### empty():在GPU创建空白tensor

In [26]:
a = torch.empty(2,3,device=device)
a

tensor([[ 0.1657, -1.1956,  1.2423],
        [ 0.0000,  0.0000,  0.0000]], device='cuda:0')

## demo

### tensor分割
将一个tensor分割成多份，存在一个batch tensor中

In [27]:
import torch
from torch.autograd import Variable
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
mother = torch.randn(32,32,3,device=device)

In [32]:
childs = torch.empty(4,8,8,3,device=device)

In [33]:
childs[0] = mother[:8,:8,:]
childs[1] = mother[8:16,8:16,:]

In [34]:
mother[0,0,-2:]

tensor([-0.6308, -0.0839], device='cuda:0')